In [30]:
import os
import itertools
from PIL import Image

# Data Handling Tools
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Deep Learning Libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras import regularizers

import warnings
warnings.filterwarnings("ignore")

In [31]:
# Generate Data Paths for Train
train_data_dir = 'D:\ML\DISEASE PREDICTION\PNEUMONIA\Dataset\Train'
filepaths = []
labels = []
folds = os.listdir(train_data_dir) # List of All directory with Train Directory, return 'PNEUMNONIA' & 'NORMAL'
for fold in folds:
    foldpath = os.path.join(train_data_dir,fold)
    fileList = os.listdir(foldpath)
    for file in fileList:
        fpath = os.path.join(foldpath,file)
        filepaths.append(fpath)
        labels.append(fold)

FSeries = pd.Series(filepaths, name='filepaths')
LSeries = pd.Series(labels, name='labels')

# Create a New DataFrame
train_df = pd.concat([FSeries, LSeries], axis=1)
train_df

,filepaths,labels
0,D:\ML\DISEASE PREDICTION\PNEUMONIA\Dataset\Tra...,NORMAL
1,D:\ML\DISEASE PREDICTION\PNEUMONIA\Dataset\Tra...,NORMAL
2,D:\ML\DISEASE PREDICTION\PNEUMONIA\Dataset\Tra...,NORMAL
3,D:\ML\DISEASE PREDICTION\PNEUMONIA\Dataset\Tra...,NORMAL
4,D:\ML\DISEASE PREDICTION\PNEUMONIA\Dataset\Tra...,NORMAL
...,...,...
1816,D:\ML\DISEASE PREDICTION\PNEUMONIA\Dataset\Tra...,PNEUMONIA
1817,D:\ML\DISEASE PREDICTION\PNEUMONIA\Dataset\Tra...,PNEUMONIA
1818,D:\ML\DISEASE PREDICTION\PNEUMONIA\Dataset\Tra...,PNEUMONIA
1819,D:\ML\DISEASE PREDICTION\PNEUMONIA\Dataset\Tra...,PNEUMONIA


In [32]:
# Generate Data Paths for Test
test_data_dir = 'D:\ML\DISEASE PREDICTION\PNEUMONIA\Dataset\Test'
filepaths = []
labels = []
folds = os.listdir(test_data_dir)
for fold in folds:
    foldpath = os.path.join(test_data_dir,fold)
    fileList = os.listdir(foldpath)
    for file in fileList:
        fpath = os.path.join(foldpath,file)
        filepaths.append(fpath)
        labels.append(fold)
FSeries = pd.Series(filepaths,name='filepaths')
LSeries = pd.Series(labels,name='labels')
test_df = pd.concat([FSeries,LSeries], axis=1)

In [33]:
# Generate Data Paths for Train
val_data_dir = 'D:\ML\DISEASE PREDICTION\PNEUMONIA\Dataset\Validation'
filepaths = []
labels = []
folds = os.listdir(val_data_dir)
for fold in folds:
    foldpath = os.path.join(val_data_dir,fold)
    fileList = os.listdir(foldpath)
    for file in fileList:
        fpath = os.path.join(foldpath,file)
        filepaths.append(fpath)
        labels.append(fold)
FSeries = pd.Series(filepaths,name='filepaths')
LSeries = pd.Series(labels,name='labels')
valid_df = pd.concat([FSeries,LSeries], axis=1)

In [34]:
# PreProcessing
batch_size = 16
img_size = (224, 224)
tr_gen = ImageDataGenerator()
ts_gen = ImageDataGenerator()
val_gen = ImageDataGenerator()
train_gen = tr_gen.flow_from_dataframe(train_df, x_col='filepaths', y_col='labels',target_size = img_size, class_mode = 'categorical', color_mode='rgb', shuffle=True, batch_size=batch_size)
test_gen = ts_gen.flow_from_dataframe(test_df, x_col='filepaths', y_col='labels',target_size = img_size, class_mode = 'categorical', color_mode='rgb', shuffle=True, batch_size=batch_size)
valid_gen = val_gen.flow_from_dataframe(valid_df, x_col='filepaths', y_col='labels',target_size = img_size, class_mode = 'categorical', color_mode='rgb', shuffle=True, batch_size=batch_size)


Found 1821 validated image filenames belonging to 2 classes.


Found 624 validated image filenames belonging to 2 classes.
Found 16 validated image filenames belonging to 2 classes.
